# 🧪 Python pytest 실습 코드

이 노트북에서는 pytest를 사용한 유닛 테스트의 실제 예제를 실행해볼 수 있습니다. 각 섹션에서 테스트 코드를 작성하고 실행하는 방법을 배워보겠습니다.

## 설치 확인

먼저 pytest가 설치되어 있는지 확인합니다. 설치되어 있지 않다면 설치합니다.

In [ ]:
!pip install pytest
!pip install pytest-sugar  # 더 보기 좋은 결과 출력을 위한 플러그인

## 1. 기본 테스트 작성하기

가장 간단한 형태의 테스트부터 시작해 봅시다. 두 숫자를 더하는 `add` 함수를 테스트하는 코드입니다.

In [ ]:
# 먼저 테스트할 함수를 정의합니다
def add(a, b):
    return a + b

# 함수 실행 테스트
print(f"2 + 3 = {add(2, 3)}")
print(f"-1 + 1 = {add(-1, 1)}")
print(f"0.5 + 0.7 = {add(0.5, 0.7)}")

In [ ]:
# 이제 add 함수를 테스트하는 함수를 작성합니다
def test_add():
    assert add(2, 3) == 5
    assert add(-1, 1) == 0
    assert add(0.5, 0.7) == 1.2

# Jupyter Notebook에서 pytest 실행
import pytest

# -v 옵션으로 상세 결과 출력
pytest.main(['-v', '-xvs', '--no-header'])

## 2. 예외 테스트하기

함수가 특정 조건에서 예외를 발생시키는지 테스트하는 방법을 알아봅시다.

In [ ]:
# 나눗셈 함수 정의
def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b

# 정상 케이스 테스트
print(f"10 / 2 = {divide(10, 2)}")

# 예외 케이스 테스트
try:
    divide(10, 0)
except ValueError as e:
    print(f"예외 발생: {e}")

In [ ]:
# 나눗셈 함수의 테스트 함수 작성
def test_divide():
    # 정상 케이스 테스트
    assert divide(10, 2) == 5.0
    assert divide(8, 4) == 2.0
    assert divide(1, 3) == pytest.approx(0.333333, abs=1e-5)  # 부동소수점 비교
    
    # 예외 케이스 테스트
    with pytest.raises(ValueError, match="Cannot divide by zero"):
        divide(10, 0)

# 테스트 실행
pytest.main(['-v', '-xvs', '--no-header'])

## 3. 매개변수화된 테스트 (Parametrized Tests)

동일한 테스트 로직을 여러 입력값에 대해 실행하는 방법입니다.

In [ ]:
# 회문(palindrome) 판별 함수
def is_palindrome(s):
    s = s.lower().replace(" ", "")
    return s == s[::-1]

# 몇 가지 예제로 테스트
examples = [
    "racecar", 
    "hello", 
    "A man a plan a canal Panama",
    "Was it a car or a cat I saw",
    "python"
]

for example in examples:
    print(f"\"{example}\" 회문인가? {is_palindrome(example)}")

In [ ]:
# 매개변수화된 테스트 작성
@pytest.mark.parametrize("input_str, expected", [
    ("racecar", True),
    ("hello", False),
    ("A man a plan a canal Panama", True),
    ("Was it a car or a cat I saw", True),
    ("python", False),
    ("12321", True),
    ("", True),  # 빈 문자열도 회문으로 간주
])
def test_is_palindrome(input_str, expected):
    assert is_palindrome(input_str) == expected

# 테스트 실행
pytest.main(['-v', '-xvs', '--no-header'])

## 4. Fixture 사용하기

Fixture는 테스트를 위한 환경을 설정하고 필요한 데이터를 제공하는 방법입니다.

In [ ]:
# 테스트할 사용자 클래스
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email
        self.posts = []
    
    def add_post(self, title, content):
        post = {
            "title": title,
            "content": content,
            "id": len(self.posts) + 1
        }
        self.posts.append(post)
        return post
    
    def get_posts(self):
        return self.posts
    
    def delete_post(self, post_id):
        for i, post in enumerate(self.posts):
            if post["id"] == post_id:
                return self.posts.pop(i)
        return None

In [ ]:
# 사용자와 게시물이 있는 fixture 정의
@pytest.fixture
def user_with_posts():
    user = User("Test User", "test@example.com")
    user.add_post("First Post", "This is my first post")
    user.add_post("Second Post", "This is my second post")
    return user

# Fixture를 사용한 테스트
def test_user_posts(user_with_posts):
    # user_with_posts fixture가 테스트에 주입됩니다
    user = user_with_posts
    
    # 초기 게시물 확인
    posts = user.get_posts()
    assert len(posts) == 2
    assert posts[0]["title"] == "First Post"
    
    # 게시물 추가
    new_post = user.add_post("Third Post", "This is my third post")
    assert len(user.get_posts()) == 3
    assert new_post["id"] == 3
    
    # 게시물 삭제
    deleted = user.delete_post(2)
    assert deleted["title"] == "Second Post"
    assert len(user.get_posts()) == 2

# 테스트 실행
pytest.main(['-v', '-xvs', '--no-header'])

## 5. Mock 객체 사용하기

외부 시스템(API, 데이터베이스 등)에 의존하는 코드를 테스트할 때 Mock 객체를 사용하는 방법입니다.

In [ ]:
from unittest.mock import patch, Mock
import requests

# 외부 API를 호출하는 함수
def get_user_data(user_id):
    # 실제로는 외부 API 호출
    response = requests.get(f"https://api.example.com/users/{user_id}")
    if response.status_code == 200:
        return response.json()
    return None

# Mock을 사용하지 않고 함수 작동 방식 설명 (실제로는 오류 발생)
print("실제 API가 없으므로 다음 코드는 실행되지 않습니다:")
print('get_user_data(1)')

In [ ]:
# Mock을 사용한 테스트
def test_get_user_data():
    # Mock response 객체 생성
    mock_response = Mock()
    mock_response.status_code = 200
    mock_response.json.return_value = {"id": 1, "name": "John Doe", "email": "john@example.com"}
    
    # requests.get 함수를 mock으로 대체
    with patch('requests.get', return_value=mock_response) as mock_get:
        data = get_user_data(1)
        
        # mock_get이 올바른 URL로 호출되었는지 확인
        mock_get.assert_called_once_with("https://api.example.com/users/1")
        
        # 반환된 데이터 확인
        assert data["name"] == "John Doe"
        assert data["email"] == "john@example.com"

# 오류 응답 테스트
def test_get_user_data_error():
    # 404 응답을 반환하는 Mock 생성
    mock_response = Mock()
    mock_response.status_code = 404
    
    with patch('requests.get', return_value=mock_response):
        data = get_user_data(999)  # 존재하지 않는 사용자
        assert data is None

# 테스트 실행
pytest.main(['-v', '-xvs', '--no-header'])

## 6. 실전 예제: 계산기 클래스 테스트

이제 모든 개념을 종합하여 계산기 클래스를 완전히 테스트해봅시다.

In [ ]:
# calculator.py
class Calculator:
    def __init__(self):
        self.result = 0
    
    def reset(self):
        self.result = 0
        return self.result
    
    def add(self, a, b=None):
        if b is None:  # 누적 계산 모드
            self.result += a
            return self.result
        return a + b
    
    def subtract(self, a, b=None):
        if b is None:  # 누적 계산 모드
            self.result -= a
            return self.result
        return a - b
    
    def multiply(self, a, b=None):
        if b is None:  # 누적 계산 모드
            self.result *= a
            return self.result
        return a * b
    
    def divide(self, a, b=None):
        if b is None:  # 누적 계산 모드
            if a == 0:
                raise ValueError("Cannot divide by zero")
            self.result /= a
            return self.result
        if b == 0:
            raise ValueError("Cannot divide by zero")
        return a / b
    
# 계산기 클래스 간단한 사용 예시
calc = Calculator()
print(f"2 + 3 = {calc.add(2, 3)}")
print(f"5 - 2 = {calc.subtract(5, 2)}")
print(f"4 * 3 = {calc.multiply(4, 3)}")
print(f"10 / 2 = {calc.divide(10, 2)}")

print("\n누적 계산 모드:")
calc.reset()
print(f"현재 값: {calc.result}")
print(f"+ 5 = {calc.add(5)}")
print(f"* 2 = {calc.multiply(2)}")
print(f"- 3 = {calc.subtract(3)}")
print(f"/ 2 = {calc.divide(2)}")

In [ ]:
# 계산기 클래스 테스트
@pytest.fixture
def calculator():
    return Calculator()

# 기본 연산 테스트
def test_calculator_operations(calculator):
    assert calculator.add(2, 3) == 5
    assert calculator.subtract(5, 2) == 3
    assert calculator.multiply(3, 4) == 12
    assert calculator.divide(10, 2) == 5.0

# 누적 계산 모드 테스트
def test_calculator_accumulation(calculator):
    assert calculator.result == 0
    
    assert calculator.add(5) == 5
    assert calculator.result == 5
    
    assert calculator.multiply(2) == 10
    assert calculator.result == 10
    
    assert calculator.subtract(3) == 7
    assert calculator.result == 7
    
    assert calculator.divide(2) == 3.5
    assert calculator.result == 3.5
    
    assert calculator.reset() == 0
    assert calculator.result == 0

# 에지 케이스 테스트
def test_calculator_edge_cases(calculator):
    # 0으로 나누기 테스트
    with pytest.raises(ValueError, match="Cannot divide by zero"):
        calculator.divide(10, 0)
    
    # 누적 모드에서 0으로 나누기
    calculator.add(10)  # result = 10
    with pytest.raises(ValueError, match="Cannot divide by zero"):
        calculator.divide(0)
    
    # 부동소수점 비교
    assert calculator.divide(10, 3) == pytest.approx(3.333333, abs=1e-5)

# 매개변수화된 테스트
@pytest.mark.parametrize("a, b, expected", [
    (1, 1, 2),  # 양수 + 양수
    (1, -1, 0),  # 양수 + 음수
    (-1, -1, -2),  # 음수 + 음수
    (0, 0, 0),  # 0 + 0
])
def test_calculator_add_parametrized(calculator, a, b, expected):
    assert calculator.add(a, b) == expected

# 테스트 실행
pytest.main(['-v', '-xvs', '--no-header'])

## 7. 테스트 마크와 테스트 건너뛰기

테스트에 마크를 지정하고 특정 테스트를 건너뛰는 방법을 알아봅시다.

In [ ]:
# 느린 테스트와 빠른 테스트 구분
import time

@pytest.mark.slow
def test_slow_operation():
    time.sleep(1)  # 시간이 오래 걸리는 작업 시뮬레이션
    assert True

@pytest.mark.fast
def test_fast_operation():
    assert 1 + 1 == 2

# 테스트 건너뛰기
@pytest.mark.skip(reason="이 테스트는 아직 구현 중입니다")
def test_unimplemented_feature():
    # 아직 개발 중인 기능
    assert False

# 조건부 건너뛰기
@pytest.mark.skipif(True, reason="특정 조건에서만 실행")
def test_conditional_skip():
    assert True

# 실패가 예상되는 테스트
@pytest.mark.xfail(reason="알려진 버그")
def test_known_bug():
    # 아직 수정되지 않은 버그 테스트
    assert 1 == 2

# 테스트 실행
pytest.main(['-v', '-xvs', '--no-header'])

## 8. 테스트 커버리지 측정하기

pytest-cov 플러그인을 사용하여 테스트 커버리지를 측정하는 방법입니다. 이 예제는 노트북에서 직접 실행하기는 어렵지만, 명령줄에서 실행하는 방법을 보여드립니다.

In [ ]:
# pytest-cov 설치
!pip install pytest-cov

커버리지 측정은 다음과 같이 명령줄에서 실행합니다:

```bash
# 기본 커버리지 측정
pytest --cov=my_module

# 자세한 보고서 생성
pytest --cov=my_module --cov-report=term-missing

# HTML 보고서 생성
pytest --cov=my_module --cov-report=html
```

## 9. pytest.ini 설정 파일 만들기

실제 프로젝트에서는 pytest.ini 파일을 사용하여 pytest의 기본 설정을 구성할 수 있습니다:

In [ ]:
%%writefile pytest.ini
[pytest]
testpaths = tests
python_files = test_*.py
python_classes = Test*
python_functions = test_*

markers =
    slow: marks tests as slow (deselect with '-m "not slow"')
    fast: marks tests as fast

addopts = -v --cov=my_module

## 10. 실전 프로젝트 구조 예시

마지막으로, 실제 프로젝트에서 테스트를 구성하는 방법을 간단히 살펴보겠습니다:

In [ ]:
# 프로젝트 구조 예시 출력
project_structure = """
my_project/
├── my_module/
│   ├── __init__.py
│   ├── calculator.py
│   └── utils.py
├── tests/
│   ├── __init__.py
│   ├── conftest.py         # 공통 fixture 정의
│   ├── test_calculator.py
│   └── test_utils.py
├── pytest.ini
├── requirements.txt
└── setup.py
"""

print(project_structure)

## 결론

이 노트북에서는 pytest를 사용하여 파이썬 코드를 효과적으로 테스트하는 방법을 실습해 보았습니다. 기본적인 테스트 작성부터 고급 기능 (fixture, mock, 매개변수화된 테스트 등)까지 다양한 예제를 통해 배웠습니다.

테스트는 코드의 품질을 유지하고 버그를 조기에 발견하는 데 필수적입니다. 적절한 테스트 전략과 pytest의 강력한 기능을 활용하면 더 견고하고 유지보수하기 쉬운 코드를 작성할 수 있습니다.

더 자세한 내용은 [pytest 공식 문서](https://docs.pytest.org/)를 참조하세요.